In [ ]:
import cv2
import numpy as np

image = cv2.imread('exercise/dalat.jpg')  
if image is None:
    print("Error: Could not load 'dalat.jpg'. Please ensure the file exists in the specified path.")
    exit()

original_rows, original_cols = image.shape[:2]
s
langbiang_crop_width = 474
langbiang_crop_height = 317

x_start = 0
y_start = 0
x_end = langbiang_crop_width
y_end = langbiang_crop_height

x_end = min(x_end, original_cols)
y_end = min(y_end, original_rows)

langbiang_only = image[y_start:y_end, x_start:x_end].copy()

if langbiang_only.size == 0:
    print("Error: LangBiang crop resulted in an empty image. Check crop coordinates.")
    exit()

langbiang_rows, langbiang_cols = langbiang_only.shape[:2]

translation_x = 100
translation_y = 0

new_cols_canvas = original_cols + translation_x
new_rows_canvas = original_rows + translation_y

expanded_image = np.zeros((new_rows_canvas, new_cols_canvas, 3), dtype=np.uint8)

expanded_image[0:original_rows, 0:original_cols] = image

M = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
translated_langbiang_on_new_canvas = cv2.warpAffine(langbiang_only, M, (new_cols_canvas, new_rows_canvas))

translated_langbiang_gray = cv2.cvtColor(translated_langbiang_on_new_canvas, cv2.COLOR_BGR2GRAY)
_, translated_mask = cv2.threshold(translated_langbiang_gray, 1, 255, cv2.THRESH_BINARY)

translated_mask_inv = cv2.bitwise_not(translated_mask)

background_roi = cv2.bitwise_and(expanded_image, expanded_image, mask=translated_mask_inv)

foreground_roi = cv2.bitwise_and(translated_langbiang_on_new_canvas, translated_langbiang_on_new_canvas, mask=translated_mask)

result = cv2.add(background_roi, foreground_roi)

cv2.imwrite('lang_biang_translated.jpg', result)
cv2.imshow('LangBiang Translated Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
#bai2
import cv2
import numpy as np

image = cv2.imread('exercise/dalat.jpg') 
if image is None:
    print("Không tìm thấy ảnh. Vui lòng kiểm tra đường dẫn.")
    exit()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

adaptive_thresh = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    61,   
    5     
)

contours, _ = cv2.findContours(adaptive_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print("Số lượng contour tìm được:", len(contours))

if len(contours) == 0:
    print("Không tìm thấy vùng nào! Hãy thử điều chỉnh blockSize hoặc C.")
    exit()

contours = sorted(contours, key=cv2.contourArea, reverse=True)
hxh_contour = contours[0]  

mask = np.zeros_like(gray)
cv2.drawContours(mask, [hxh_contour], -1, 255, -1)

hxh_region = cv2.bitwise_and(image, image, mask=mask)

(h, w) = hxh_region.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated = cv2.warpAffine(hxh_region, M, (w, h))

cv2.imwrite('ho_xuan_huong.jpg', rotated)
cv2.imshow("Ho Xuan Huong result", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()


Số lượng contour tìm được: 1


In [3]:
#3
import cv2
import numpy as np


try:
    img = cv2.imread('exercise/dalat.jpg')

    if img is None:
        print(f"Không thể đọc ảnh từ đường dẫn: {image_path}")
    else:
        y_start = 0
        y_end = img.shape[0] // 2 - 50 
        x_start = img.shape[1] // 2 + 50 
        x_end = img.shape[1]

        quan_truong_lam_vien_roi = img[y_start:y_end, x_start:x_end]

        if quan_truong_lam_vien_roi.size == 0:
            print("Vùng chọn trống. Vui lòng kiểm tra lại tọa độ.")
        else:
            gray_roi = cv2.cvtColor(quan_truong_lam_vien_roi, cv2.COLOR_BGR2GRAY)

            _, binary_roi = cv2.threshold(gray_roi, 128, 255, cv2.THRESH_BINARY_INV)
            kernel = np.ones((5, 5), np.uint8)

            closed_roi = cv2.morphologyEx(binary_roi, cv2.MORPH_CLOSE, kernel)

            output_filename = 'quan_truong_lam_vien.jpg'
            cv2.imwrite(output_filename, closed_roi)

            print(f"Đã xử lý và lưu ảnh Quảng trường Lâm Viên vào {output_filename}")

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

Đã xử lý và lưu ảnh Quảng trường Lâm Viên vào quan_truong_lam_vien.jpg


In [1]:
def main_menu():
    """Hiển thị menu chính và xử lý lựa chọn của người dùng."""
    print("=== MENU CHÍNH ===")
    print("1. Biến đổi hình học (geometric_transformation):")
    print("   - coordinate_mapping")
    print("   - rotate")
    print("   - scale")
    print("   - shift")
    print("2. Phân đoạn ảnh (segment):")
    print("   - adaptive_thresholding")
    print("   - binary_dilation")
    print("   - binary_erosion")
    print("   - otsu")
    print("\nNhập 'exit' để thoát chương trình.")

    while True:
        user_input = input("\nNhập chức năng (ví dụ: 'rotate' hoặc 'segment binary_dilation'): ").strip().lower()

        if user_input == 'exit':
            print("✅ Đang thoát chương trình.")
            break

        parts = user_input.split(' ', 1)

        if len(parts) == 1:
            function = parts[0]
            execute_single_function(function)
        elif len(parts) == 2:
            category, sub_function = parts
            execute_combined_function(category, sub_function)
        else:
            print("❌ Lựa chọn không hợp lệ. Vui lòng thử lại.")

def execute_single_function(function):
    """Xử lý khi người dùng chỉ nhập một chức năng cụ thể."""
    if function in ['coordinate_mapping', 'rotate', 'scale', 'shift']:
        print(f"🔄 Thực hiện Biến đổi hình học: {function.capitalize()}...")
    elif function in ['adaptive_thresholding', 'binary_dilation', 'binary_erosion', 'otsu']:
        print(f"🧪 Thực hiện Phân đoạn ảnh: {function.replace('_', ' ').title()}...")
    else:
        print("❌ Chức năng không hợp lệ. Vui lòng thử lại.")

def execute_combined_function(category, sub_function):
    """Xử lý khi người dùng nhập nhóm + chức năng con."""
    if category == 'geometric_transformation':
        if sub_function in ['coordinate_mapping', 'rotate', 'scale', 'shift']:
            print(f"🔄 Thực hiện Biến đổi hình học: {sub_function.capitalize()}...")
        else:
            print("❌ Chức năng con không hợp lệ cho geometric_transformation.")
    elif category == 'segment':
        if sub_function in ['adaptive_thresholding', 'binary_dilation', 'binary_erosion', 'otsu']:
            print(f"🧪 Thực hiện Phân đoạn ảnh: {sub_function.replace('_', ' ').title()}...")
        else:
            print("❌ Chức năng con không hợp lệ cho segment.")
    else:
        print("❌ Nhóm chức năng không hợp lệ. Vui lòng thử lại.")

if __name__ == "__main__":
    main_menu()


=== MENU CHÍNH ===
1. Biến đổi hình học (geometric_transformation):
   - coordinate_mapping
   - rotate
   - scale
   - shift
2. Phân đoạn ảnh (segment):
   - adaptive_thresholding
   - binary_dilation
   - binary_erosion
   - otsu

Nhập 'exit' để thoát chương trình.
❌ Chức năng không hợp lệ. Vui lòng thử lại.
🔄 Thực hiện Biến đổi hình học: Rotate...
✅ Đang thoát chương trình.
